In [ ]:

# Step 2: Import Necessary Libraries
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML

# Step 3: Create an Interactive Widget (Slider + Graph)
def plot_graph(frequency=1):
    x = np.linspace(0, 10, 100)
    y = np.sin(frequency * x)
    
    plt.figure(figsize=(6, 4))
    plt.plot(x, y, label=f'Sine Wave (freq={frequency})', color='blue')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.title('Interactive Sine Wave')
    plt.legend()
    plt.grid(True)
    plt.show()

slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.1, description='Frequency:')
ui = widgets.VBox([slider])
out = widgets.interactive_output(plot_graph, {'frequency': slider})

display(HTML('<h2 style="color: green;">Voila Interactive Graph Demo</h2>'))
display(ui, out)

# Step 4: Display a Button and Textbox Example
def on_button_click(b):
    output_text.value = f'Hello, {name_input.value}! Voila is amazing!'

name_input = widgets.Text(placeholder='Enter your name', description='Name:')
button = widgets.Button(description='Greet Me!')
output_text = widgets.Label()
button.on_click(on_button_click)

display(HTML('<h3>Interactive Button and Textbox</h3>'))
display(name_input, button, output_text)

# Step 5: Embed HTML & Styling
display(HTML('<h3 style="color: blue;">You can embed HTML, CSS, and JavaScript in Voila!</h3>'))
display(HTML('<p style="font-size:16px;">Try running this notebook with Voila:</p>'))
display(HTML('<code>voila voila_demo.ipynb</code>'))